In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import date
import calendar


YEAR = 2018
MONTH = [1,2,3,4,5,6,7,8,9,10,11,12]
MONTH_END_DATE_DASH = ["2018-01-31",
                       "2018-02-28",
                       "2018-03-31",
                       "2018-04-30",
                       "2018-05-31",
                       "2018-06-30",
                       "2018-07-31",
                       "2018-08-31",
                       "2018-09-30",
                       "2018-10-31",
                       "2018-11-30",
                       "2018-12-31"]


def get_distinct_tier_list(employer_master):
    '''
        This function will list the type of tier
    '''
    return list(set(employer_master["Tier"]))


def get_distinct_employer_no_list(employer_master):
    '''
        This function will list the distinct Employer_No
    '''
    return list(set(employer_master["Employer_No"]))


def is_new_account_for_month(formatted_effective_from, month_end_date):
    '''
        This function will check whether the new account is in this month
    '''
    if datetime.strptime(month_end_date, "%d/%m/%Y").strftime("%Y") != datetime.strptime(effective_from, "%d/%m/%Y").strftime("%Y"):
        return False
    return datetime.strptime(month_end_date, "%d/%m/%Y").strftime("%m") == datetime.strptime(effective_from, "%d/%m/%Y").strftime("%m")


def tier_list(employer_master):
    '''
        This function will generate the Tier column in the output
    '''
    distinct_tier_list = get_distinct_tier_list(employer_master)
    tier_list = []
    for i in distinct_tier_list:
        for j in range(len(MONTH)):
            tier_list.append(i)
    return tier_list


def month_end_date_list(employer_master):
    '''
        This function will generate the Month End Date column in the output
    '''
    distinct_tier_list = get_distinct_tier_list(employer_master)
    month_end_date_list = []
    month_end_date = []
    for month in MONTH:
        weekDay,monthCountDay = calendar.monthrange(YEAR,month)
        lastDay = date(YEAR,month,day = monthCountDay)
        month_end_date.append(lastDay.strftime("%d/%m/%Y"))
    for i in range(len(distinct_tier_list)):
        for j in month_end_date:
            month_end_date_list.append(j)
    return month_end_date_list
    


def payments(employer_master, payment_transactions):
    '''
        This function will generate the Num Payments column and Amount Of Payments column in the output
    '''
    num_payments_list = []
    amount_of_payments_list = []
    for i in get_distinct_tier_list(employer_master):
        tier = employer_master.loc[employer_master["Tier"] == i]
        employer_num_list = list(set(tier["Employer_No"]))
        transactions = payment_transactions.loc[payment_transactions["Employer_No"].isin(employer_num_list)]
        transactions['Formatted_Cash_Received_Date'] = pd.to_datetime(transactions['Cash_Received_Date'], format = '%d/%m/%Y').dt.date
        for month in MONTH:
            weekDay,monthCountDay = calendar.monthrange(YEAR,month)
            firstDay = date(YEAR,month,day = 1)
            lastDay = date(YEAR,month,day = monthCountDay)
            transaction_monthly = transactions.loc[(transactions["Formatted_Cash_Received_Date"] >= firstDay) & (transactions["Formatted_Cash_Received_Date"] <= lastDay)]
            num_payments_list.append(len(transaction_monthly))
            amount_of_payments_list.append("%.2f" % transaction_monthly["Total_Amt"].sum())
    return num_payments_list, amount_of_payments_list


def new_employers_list(employer_master):
    '''
        This function will generate the New Employers column in the output
    '''
    new_employers_count_list_per_month = []
    distinct_tier_list = get_distinct_tier_list(employer_master)
    distinct_employer_list = get_distinct_employer_no_list(employer_master)
    for tier in distinct_tier_list:
        for eom in MONTH_END_DATE_DASH:
            year_month = eom.rsplit("-", 1)[0]
            count = 0
            for employer in distinct_employer_list:
                records = employer_master.loc[
                    (employer_master["Tier"] == tier) & 
                    (employer_master["Effective_From"] <= eom) & 
                    (employer_master["Employer_No"] == employer)
                ]
                
                first_record_prior_to_this_month = records.head(1).loc[records.head(1)["Effective_From"] < year_month]
                if not records.empty and first_record_prior_to_this_month.empty:
                    count = count + 1
            new_employers_count_list_per_month.append(count)
    return new_employers_count_list_per_month


def open_accounts_at_eom_list(employer_master):
    '''
        This function will generate the Open Accounts At EOM column in the output
    '''
    distinct_tier_list = get_distinct_tier_list(employer_master)
    open_account_count_list_per_eom = []
    for tier in distinct_tier_list:
        for eom in MONTH_END_DATE_DASH:
            count = len(employer_master.loc[
                (employer_master["Tier"] == tier) & 
                (employer_master["Effective_From"] <= eom) & 
                (employer_master["Effective_To"] >= eom) &
                (employer_master["Status"] == "Open")
             ])
            open_account_count_list_per_eom.append(count)
    return open_account_count_list_per_eom


def main():
    '''
        This function will generate the output
    '''
    employer_master = pd.read_csv("Employer master.csv")
    payment_transactions = pd.read_csv("Payment transactions.csv")
    data = {"Tier": tier_list(employer_master),
           "Month end date": month_end_date_list(employer_master),
           "Num payments":payments(employer_master, payment_transactions)[0],
           "Amount of payments": payments(employer_master, payment_transactions)[1],
           "New employers": new_employers_list(employer_master),
           "Open employers at EOM": open_accounts_at_eom_list(employer_master)}
    df = pd.DataFrame(data)
    df.to_csv('results.csv')

    
if __name__ == "__main__":
    main()


/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
